### Import all necesaary packages


In [2]:
from requests import session
from bs4 import BeautifulSoup as bs
import re
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException
from selenium.common.exceptions import NoSuchElementException
from urllib.parse import urlparse
from requests.exceptions import HTTPError
import mysql.connector


### Find all API categories


In [3]:
API_URL = 'https://rapidapi.com/categories'
with session() as s:
    r1 = s.get(API_URL)

soup = bs(r1.text, 'html.parser')
htmlText = soup.prettify()
API_Categories= soup.find_all('div',{'CollectionCarousel CollectionCarousel'})

### Get all API category links

In [4]:
# Parse the HTML content
soup_all_categories = bs(r1.text, 'html.parser')

# Find the <a> tag with the specified class
a_tags = soup.find_all('a', class_='CardLink')
href_values = []
categories = []
if a_tags:
    for a_tag in a_tags:
        href_value = a_tag.get('href')
        href_values.append(href_value)
        category = a_tag.text.strip()
        categories.append(category)      
else:
    print('No match found.')

No match found.


In [ ]:
if href_values:
    href_value = ["https://rapidapi.com" + href for href in href_values]    
    api_href_pairs = list(zip(categories, href_value))
    print(api_href_pairs)
else:
    print('No href value found.')

### Get API links for each category link


In [ ]:
options = webdriver.ChromeOptions()
options.add_argument('--headless')  # Run Chrome in headless mode, without opening a window

user_data_dir = r'C:\Users\xxx\AppData\Local\Google\Chrome\User Data'

webdriver_path = 'D:\chromedriver_win32\chromedriver.exe'  # Replace with the actual path to the chromedriver executable

options.add_argument(f'--user-data-dir={user_data_dir}')

driver = webdriver.Chrome(executable_path=webdriver_path, options=options)

#for x in href_value:
    #print(x)
    # Load the initial URL
driver.get('https://rapidapi.com/category/News,%20Media')
base_urls = []
def scrape_page(url):
    soup = bs(driver.page_source, 'html.parser')
    a_tags = soup.find_all('a', class_='CardLink')
    if a_tags:
        for a_tag in a_tags:
            base_url = "https://rapidapi.com" + a_tag.get('href')
            base_urls.append(base_url)
def scrape_all_pages(base_url):
     while True:
        scrape_page(driver.current_url)
        next_button = None
        try:
            next_button = WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.XPATH, "//button[contains(@class, 'r-page-link') and contains(text(),'›') and not(@disabled)]")))
            driver.execute_script("arguments[0].click();", next_button)
            WebDriverWait(driver, 10).until(EC.staleness_of(next_button))
        except:
            if next_button is None:
                break

scrape_all_pages('https://rapidapi.com/category/News,%20Media')
driver.quit()  # Close the browser after scraping    

# Print the scraped URLs
for url in base_urls:
    print(url)


### Get Details from About Page (Followers,Resources URL, Ratings, Votes)


In [ ]:
about_details = {}
for base_url in base_urls:
    try:
        resource_url=''
        r3 = s.get(base_url+"details")
        r3.raise_for_status()  # Raise an exception for 4xx or 5xx status codes
        soup3 = bs(r3.text, 'html.parser')
        sections = soup3.find_all('section')

        # Iterate over each section
        for section in sections:
            followers_div = section.find('div', class_='body1 bold')
            if followers_div:
                followers_text = followers_div.text.strip()
                if followers_text.startswith('Followers:'):
                    followers = followers_text[len('Followers:'):].strip()
           
            resources_div = section.find('div', class_='body1 bold')
            #resource_url = ''
            if resources_div and 'Resources:' in resources_div.text:
                resource_url = section.find('a')['href']
        find_ratings= soup3.find('div', class_='ProductRatings')
        if find_ratings:
            ratings = find_ratings.get_text(strip=True)
        else:
            ratings = ''
        about_details[base_url] = {
            'followers': followers,
            'resourses': resource_url,
            'ratings_votes': ratings
        }
    except HTTPError as e:
        if e.response.status_code == 404:
            continue
    except NoSuchElementException:
        continue 
print(about_details)

### Upload to Table aboutPage

In [ ]:

# Connect to MySQL
cnx = mysql.connector.connect(
    host='xxx',
    user='xxx',
    password='xxx',
    database='xxx'
)

cursor = cnx.cursor()

# Prepare the SQL statement for the insert operation
insert_about_details = "INSERT INTO aboutPage (base_url, followers, resources, ratings_votes) VALUES (%s, %s, %s, %s)"

# Iterate over the items in the api_details dictionary
for base_url, details in about_details.items():
    followers = details['followers']
    resourses = details['resourses']
    ratings_votes = details['ratings_votes']

    # Check if URL already exists in the database
    select_query = "SELECT base_url FROM aboutPage WHERE base_url = %s"
    cursor.execute(select_query, (base_url,))
    existing_url = cursor.fetchone()

    if existing_url:
        print(f"URL {base_url} already exists in the database. Skipping insertion.")
    else:
        # Execute the insert operation for each URL and details
        cursor.execute(insert_about_details, (base_url, followers, resourses, ratings_votes))

# Commit the changes and close the connection
cnx.commit()
cnx.close()

### Get Documentation word count length for each API

In [ ]:
# Connect to MySQL
cnx = mysql.connector.connect(
    host='xxx',
    user='xxx',
    password='xxx',
    database='xxx'
)

cursor = cnx.cursor()
select_query = "SELECT base_url FROM aboutPage"

cursor.execute(select_query)

# Fetch all the results
results = cursor.fetchall()

# Extract base_urls from results
base_urls = [row[0] for row in results]

# Close cursor and connection
cursor.close()
cnx.close()

In [ ]:
base_url_div = {}

for base_url in base_urls:
    r4 = s.get(base_url + "details")


    soup4 = bs(r4.text, 'html.parser')
    Documentation = []
    Documentation = soup4.find_all("div", class_="MarkdownPreview")
    word_count = 0

    for div in Documentation:
        text = div.get_text()  # Get the text content within the div
        words = text.split()  # Split the text into words
        word_count = len(words)  # Count the number of words
    base_url_div[base_url] = word_count


### Update aboutPage table

In [ ]:
# Establishing a connection to the MySQL database
cnx = mysql.connector.connect(
    host='xxx',
    user='xxx',
    password='xxx',
    database='xxx'
)
cursor = cnx.cursor()

# Iterating through base_url_div dictionary and updating database
for base_url, div_value in base_url_div.items():
    update_query = f"UPDATE aboutPage SET documentation = '{div_value}' WHERE base_url = '{base_url}'"
    cursor.execute(update_query)

# Committing changes and closing cursor and connection
cnx.commit()
cursor.close()
cnx.close()
